# Combining Excel and Python

In this notebook we will explore combining Excel and Python. We will focus on using Python to drive Excel, though it is also possible to run Python code from inside Excel, but outside the scope of this class.

## Required Imports

We just need the `xlwings` package for this, and the convention is to import as `xw`. Let's also go ahead and import `xlwings` as well, because `xlwings` has an integration with `pandas`.

In [1]:
import xlwings as xw
import pandas as pd

## Connect to Excel

> Note: The process described here is one of several ways to accomplish this, but this is the most organized and consistent way.

So that we can start writing to and reading values from Excel, we need to first connect to the workbook. Then from the workbook, we select the worksheet we want to work on. Then we can read and write values. Both the workbook and worksheet are represented by types of objects in Python.

### Connect to the Workbook

We use `xw.Book` to look up the workbook and open it if necessary. You pass the name (or full file path) of the workbook including the extension.

In [2]:
book = xw.Book('Example Workbook.xlsx')

In [3]:
book

<Book [Example Workbook.xlsx]>

In [4]:
type(book)

xlwings.main.Book

We can see it's a custom `xlwings` type for the workbook. We can also see that we have the correct workbook.

### Connect to the Worksheet

Now that we have the workbook, we can look up the appropriate sheet. On the book object we already created, we can access `.sheets` to look up the sheets.

In [5]:
book.sheets

Sheets([<Sheet [Example Workbook.xlsx]Sheet1>])

If you just look at the sheets object, it will show the sheets available in the workbook. You can look up from this sheets object like either a list (integer indexing) or like a dictionary (by name).

In [6]:
sht = book.sheets[0]
sht

<Sheet [Example Workbook.xlsx]Sheet1>

In [7]:
sht = book.sheets['Sheet1']
sht

<Sheet [Example Workbook.xlsx]Sheet1>

In [8]:
type(sht)

xlwings.main.Sheet

We can see whether we look up by name or position, the result is equivalent. In general I would recommend using the name if it is a useful name, and by position if you are just using the default sheet name, but your particular use case may vary.

### All in One

Just as a quick copy-pastable snippet, here's the above in one cell:

In [9]:
book = xw.Book('Example Workbook.xlsx')
sht = book.sheets['Sheet1']

## Read and Write Values

Now that we have the sheet object, we can access values in the worksheet or write values to it. All of these just use the `.range().value` of the sheet object. In `.range()` we will pass the cell range, such as `.range('B4')` or `.range('B4:C6')`. Then adding `.value` on the end says we want to read or write the value. 

> It is also possible to write or read the formula with `.formula`, but that's outside the scope of this class.

### Reading a Single Value

Let's read the value which is in cell A1 into Python.

In [10]:
sht.range('A1').value

'Something'

We can see we got the string 'Something' because that was what was in A1 in Excel.

In [11]:
sht.range('C1').value is None

True

We will get None if there is no value in the cell.

### Writing a Single Value

The only difference from writing the value is now we assign to `.value`.

In [12]:
sht.range('A2').value = 10

Now in the Excel worksheet, you should see 10 in cell A2.

### Reading Multiple Explicit Values

The only difference for reading multiple values is that you write the cell range instead of a single cell reference inside `.range()`, and we get a list of values instead of a single value.

In [13]:
sht.range('A1:A2').value

['Something', 10.0]

In [14]:
sht.range('A1:B1').value

['Something', 'Else']

Something different happens once we select a two-dimensional range (more than one row, more than one column). Then the results come as a list of lists, where each inner list is a row in Excel.

In [15]:
sht.range('A1:B2').value

[['Something', 'Else'], [10.0, None]]

You may not want to work with this list of lists structure, so you can put it in a DataFrame instead. We can do this by adding `.options(pd.DataFrame)` onto the end of `.range()` and before `.value`.

In [16]:
sht.range('A1:B2').options(pd.DataFrame).value

,Else
Something,
10.0,None


By default, it will assume that your data has both column names and an index. You can turn either of these off in the options.

In [17]:
sht.range('A1:B2').options(pd.DataFrame, index=False).value

,Something,Else
0,10.0,None


In [18]:
sht.range('A1:B2').options(pd.DataFrame, header=False).value

,0
Something,Else
10.0,None


In [19]:
sht.range('A1:B2').options(pd.DataFrame, index=False, header=False).value

,0,1
0,Something,Else
1,10,None


### Writing Multiple Values Explicity

Again this is just the prior section in reverse. Here you can either assign a single value and have it come to each cell, or pass a data structure matching the shape of the cell area to have it put individual values in the cells.

In [20]:
sht.range('D1:D2').value = 5

In [21]:
sht.range('A5:B5').value = [10, 11]

In [22]:
sht.range('A7:B8').value = [[12, 13], [14, 15]]

### Reading an Entire Row, Column, or Table

There is the `.expand` option which allows referencing a single cell, then continuing to the right, down, or both until it hits blank cells. This is useful if the data size can be changing or you're not sure in advance how large it will be.

Passing `'right'` to `.expand` means it will start from the cell and get everything to the right.

In [23]:
sht.range('A7').expand('right').value

[12.0, 13.0]

Passing `'down'` to `.expand` means it will start from the cell and get everything downwards.

In [24]:
sht.range('A7').expand('down').value

[12.0, 14.0]

Finally, not passing any options takes the default of `'table'`, which expands both right and down.

In [25]:
sht.range('A7').expand().value

[[12.0, 13.0], [14.0, 15.0]]

It is still possible to pass options to convert to a DataFrame.

In [26]:
sht.range('A7').expand().options(pd.DataFrame, index=False, header=False).value

,0,1
0,12.0,13.0
1,14.0,15.0


### Writing an Entire Table

If we create a DataFrame, then we can write the entire DataFrame into the workbook, starting at a certain cell.

In [27]:
df = pd.DataFrame([[16, 17], [18, 19]])
df

,0,1
0,16,17
1,18,19


Just like with reading, by default it will include the columns and index in Excel.

In [28]:
sht.range('A10').value = df

But we can pass the same options to customize this behavior.

In [29]:
sht.range('A14').options(pd.DataFrame, index=False, header=False).value = df

### A Gotcha with Integers

If you try to read an integer value from Excel, it will come in as a `float`. This means you won't be able to loop over it out of the box. You'll just have to convert it to an `int`.

In [30]:
num_loops = sht.range('F5').value
num_loops

5.0

In [31]:
for i in range(num_loops):
    print('woo')

TypeError: 'float' object cannot be interpreted as an integer

As we've seen before, you can't loop over a float. So we can just wrap it in `int()` to conver it to an `int`.

In [32]:
num_loops = int(num_loops)
for i in range(num_loops):
    print('woo')

woo
woo
woo
woo
woo


### Reading Numbers with Formatting

If you read a number with currency or accounting format, it will come in as a Decimal object. 

In [33]:
my_num = sht.range('F1').value  # F1 is formatted as currency in Excel
my_num

Decimal('10.25')

This causes a problem when you try to do math with it.

In [34]:
my_num + 5.56

TypeError: unsupported operand type(s) for +: 'decimal.Decimal' and 'float'

Similarly to the last section, we can just convert to a float.

In [35]:
my_num = float(my_num)
my_num + 5.56

15.809999999999999

You can see `F3` which has accounting formatting has the same characteristic.

In [36]:
sht.range('F3').value

Decimal('10.25')

Percentages work just fine though, being converted to a float.

In [37]:
sht.range('F2').value

0.52

Dates will come in as `datetime` type, which we haven't worked with in this class, but is standard and plays well with `pandas`.

In [38]:
sht.range('F4').value

datetime.datetime(2000, 1, 1, 0, 0)

### Recalculate the Workbook

> Note: It is usually not necessary to recalculate the workbook to get your results. Changing the inputs is enough to recalculate it. Recalculating is particularly useful for internal randomness models where you want to run it multiple times with the same inputs. 

I'll first add a `=RAND()` call to the Excel workbook so we can see it being recalculated.

When we recalculate, it is at the application or workbook level and not at the cell range level. So we need to interact with `xlwings` a bit differently. First we need to get access to the application object. Here is an easy one-liner to do this:

In [65]:
app = list(xw.apps)[0]

Now that we have access to the application object, we can recalculate the workbook.

In [74]:
app.api.CalculateFull()

Notice that the `=RAND()` value is changing with each call to `CalculateFull`.

Some of you may have tried this only to see it did not work. This is because the API is a little bit different on Mac versus Windows. If the previous did not work for you, then try:

In [ ]:
api.api.calculate()

And if we want to create some code which runs on either operating system, then we can use `sys.platform` to check it:

In [ ]:
import sys

def recalculate_workbook():
    """
    Recalculates all open Excel workbooks. 
    
    Takes into account differences between operating systems to support both Windows and Mac.
    """
    app = list(xw.apps)[0]
    if sys.platform == 'win32':
        # Windows
        app.api.CalculateFull()
    elif sys.platform == 'darwin':
        # Mac OS X
        app.api.calculate()
    else:
        raise ValueError(f'unsupported OS: {sys.platform}')

Feel free to paste this function into your project and use it directly. You will also need to add the `sys` import.